In [1]:
from transformers import SiglipForImageClassification, AutoImageProcessor
import os
import pandas as pd
from PIL import Image
from datasets import Dataset
from transformers import Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch

2025-05-16 13:05:13.326381: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747400713.559701      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747400713.627315      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
model = SiglipForImageClassification.from_pretrained("prithivMLmods/Human-Action-Recognition")
processor = AutoImageProcessor.from_pretrained("prithivMLmods/Human-Action-Recognition")

config.json:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/372M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
# Donwload annotations csv from github
#! wget https://raw.githubusercontent.com/mirklys/medication-intake-detection-master-challenge/adem/frames_annotations.csv

In [3]:
! apt-get install -y gdown

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package gdown


In [4]:
# download zip from google drive and unzip contents
! gdown --id 1rbISVuHbT_AJPw4wv4ywLym3TLVuGLDH      

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1rbISVuHbT_AJPw4wv4ywLym3TLVuGLDH
From (redirected): https://drive.google.com/uc?id=1rbISVuHbT_AJPw4wv4ywLym3TLVuGLDH&confirm=t&uuid=8fa2d915-7f3b-4df4-95c3-96ec0811ae14
To: /kaggle/working/frames_and_annotations.zip
100%|███████████████████████████████████████| 1.14G/1.14G [00:11<00:00, 100MB/s]


In [5]:
# Unzip the file
import zipfile
with zipfile.ZipFile("./frames_and_annotations.zip", "r") as zip_ref:
    zip_ref.extractall("frames")


In [10]:
df = pd.read_csv("./frames/frames_annotations.csv")
# fix backslash to forward slash in relative paths
df["image"] = df["image"].apply(lambda x: os.path.join("frames", x.replace("\\", "/")))

#assign labels
df["label"] = df["label"].map({"not_taking_medication": 0, "taking_medication": 1})

# generate dataset from df
dataset = Dataset.from_pandas(df)

def preprocess(example):
    image = Image.open(example["image"]).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    inputs["labels"] = example["label"]
    return {
        "pixel_values": inputs["pixel_values"].squeeze(),
        "labels": inputs["labels"]
    }

dataset = dataset.map(preprocess, remove_columns=["image", "label"])

                                           image                  label
0    frames\video_20250328_011625_frame_0001.png  not_taking_medication
1    frames\video_20250328_011625_frame_0002.png  not_taking_medication
2    frames\video_20250328_011625_frame_0003.png  not_taking_medication
3    frames\video_20250328_011625_frame_0004.png  not_taking_medication
4    frames\video_20250328_011625_frame_0005.png      taking_medication
..                                           ...                    ...
845  frames\video_20250419_185933_frame_0006.png  not_taking_medication
846  frames\video_20250419_185933_frame_0007.png  not_taking_medication
847  frames\video_20250419_185933_frame_0008.png  not_taking_medication
848  frames\video_20250419_185933_frame_0009.png  not_taking_medication
849  frames\video_20250419_185933_frame_0010.png  not_taking_medication

[850 rows x 2 columns]


Map:   0%|          | 0/850 [00:00<?, ? examples/s]

In [12]:
dataset = dataset.train_test_split(test_size=0.1)

AttributeError: 'DatasetDict' object has no attribute 'train_test_split'

In [13]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    
    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

In [14]:
model.config.label2id = {"not_taking_medication": 0, "taking_medication": 1}
model.config.id2label = {0: "not_taking_medication", 1: "taking_medication"}

training_args = TrainingArguments(
    output_dir="./HAR-medication-finetuned",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    hub_model_id="Adekiii/HAR-medication-finetuned",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics,
)

In [26]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.735300,0.624664,0.870588,0.000000,0.000000,0.000000
2,0.455000,0.388687,0.870588,0.000000,0.000000,0.000000
3,0.512700,0.422446,0.870588,0.000000,0.000000,0.000000


c:\Users\ademk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ademk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ademk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

TrainOutput(global_step=288, training_loss=0.5537669054336019, metrics={'train_runtime': 1014.3531, 'train_samples_per_second': 2.263, 'train_steps_per_second': 0.284, 'total_flos': 1.9223958439719936e+17, 'train_loss': 0.5537669054336019, 'epoch': 3.0})

In [15]:
trainer.save_model("./HAR-med-finetuned")
processor.save_pretrained("./HAR-med-finetuned")

['./HAR-med-finetuned/preprocessor_config.json']

In [16]:
from huggingface_hub import HfApi

access_token = "token_here"

api = HfApi(token=access_token)
api.upload_folder(
    folder_path="HAR-med-finetuned",
    repo_id="Adekiii/HAR-medication-finetuned", # change with own repo
    repo_type="model",
)

HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/models/Adekiii/HAR-medication-finetuned/preupload/main (Request ID: Root=1-682739ec-4f3d41ff2680fe314fb03966;e998ca62-e44f-44f6-850f-4d70a1ca6d2e)

Invalid credentials in Authorization header